In [1]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

In [2]:
parser = argparse.ArgumentParser(description='Sequence Modeling - (Permuted) Sequential MNIST')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--cuda', action='store_false',
                    help='use CUDA (default: True)')
parser.add_argument('--dropout', type=float, default=0.05,
                    help='dropout applied to layers (default: 0.05)')
parser.add_argument('--clip', type=float, default=-1,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--epochs', type=int, default=20,
                    help='upper epoch limit (default: 20)')
parser.add_argument('--ksize', type=int, default=7,
                    help='kernel size (default: 7)')
parser.add_argument('--levels', type=int, default=8,
                    help='# of levels (default: 8)')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--lr', type=float, default=2e-3,
                    help='initial learning rate (default: 2e-3)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
parser.add_argument('--nhid', type=int, default=25,
                    help='number of hidden units per layer (default: 25)')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--permute', action='store_true',
                    help='use permuted MNIST (default: false)')
args = parser.parse_args()

torch.manual_seed(args.seed)

usage: ipykernel_launcher.py [-h] [--batch_size N] [--cuda]
                             [--dropout DROPOUT] [--clip CLIP]
                             [--epochs EPOCHS] [--ksize KSIZE]
                             [--levels LEVELS] [--log-interval N] [--lr LR]
                             [--optim OPTIM] [--nhid NHID] [--seed SEED]
                             [--permute]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/rysul/Library/Jupyter/runtime/kernel-4e1d6b14-5258-4b4b-814b-3788c2127a39.json


SystemExit: 2

/Users/rysul/opt/anaconda3/envs/torchEnv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:41].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[41] = np.where(df[41]=='normal', 'normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[41]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [96]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NSLKDDDataset(Dataset):
    def __init__(self, fileName):
        self.data = pd.read_csv(fileName, header = None)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :41].values)
        stringLower = string[0].lower()
        xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        xMid = np.array(xData)
        xArray = xMid.transpose() # should be now (39, 200)
        
        # prepate y data
        #self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        yArray = np.where(self.data.iloc[idx, 41]=='normal', 0, 1)
        
        #yArray = Ydf.to_numpy()
    
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        
        return torch.from_numpy(xArray), torch.from_numpy(yArray) # returns torch tensor of x and y

In [97]:
params = {'batch_size': 64}
fileNameTrain = 'KDDTrain+.csv'
fileNameTest = 'KDDTest+.csv'
datasetTrain = NSLKDDDataset(fileNameTrain)
datasetTest = NSLKDDDataset(fileNameTest)
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)


In [98]:
for data, target in dataGeneratorTrain:
    print(target)

tensor([0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0], dtype=torch.int32)
tensor([1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0], dtype=torch.int32)
tensor([0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0,
        1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0], dtype=torch.int32)
tensor([1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,
        0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1], dtype=torch.int32)
tensor([0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 

KeyboardInterrupt: 

In [99]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 39
seq_length = int(200)
epochs = 100
steps = 0

In [100]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6#hidden nodes times levels 
kernel_size = 5
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.05)


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

In [104]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        # print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        #loss1 = torch.nn.CrossEntropyLoss()
        loss = F.nll_loss(output, target) # negative log likelihood
        #loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        if batch_idx > 0 and batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(dataGeneratorTrain.dataset),
                100. * batch_idx / len(dataGeneratorTrain), train_loss.item()/100, steps))
            train_loss = 0

In [70]:
def test():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorTest.dataset),
            100. * correct / len(dataGeneratorTest.dataset)))
        return test_loss

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (


In [112]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [6400/32259 (20%)]	Loss: 0.695967	Steps: 148800
Train Epoch: 1 [12800/32259 (40%)]	Loss: 0.688769	Steps: 168800
Train Epoch: 1 [19200/32259 (59%)]	Loss: 0.686424	Steps: 188800
Train Epoch: 1 [25600/32259 (79%)]	Loss: 0.686073	Steps: 208800
Train Epoch: 1 [32000/32259 (99%)]	Loss: 0.685193	Steps: 228800


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.6975, Accuracy: 3594/8496 (42%)

Train Epoch: 2 [6400/32259 (20%)]	Loss: 0.688340	Steps: 249800
Train Epoch: 2 [12800/32259 (40%)]	Loss: 0.678877	Steps: 269800
Train Epoch: 2 [19200/32259 (59%)]	Loss: 0.672409	Steps: 289800
Train Epoch: 2 [25600/32259 (79%)]	Loss: 0.666752	Steps: 309800
Train Epoch: 2 [32000/32259 (99%)]	Loss: 0.656519	Steps: 329800

Test set: Average loss: 0.6686, Accuracy: 4566/8496 (54%)

Train Epoch: 3 [6400/32259 (20%)]	Loss: 0.644280	Steps: 350800
Train Epoch: 3 [12800/32259 (40%)]	Loss: 0.612639	Steps: 370800
Train Epoch: 3 [19200/32259 (59%)]	Loss: 0.569506	Steps: 390800
Train Epoch: 3 [25600/32259 (79%)]	Loss: 0.504351	Steps: 410800
Train Epoch: 3 [32000/32259 (99%)]	Loss: 0.401503	Steps: 430800

Test set: Average loss: 0.4385, Accuracy: 6708/8496 (79%)

Train Epoch: 4 [6400/32259 (20%)]	Loss: 0.296397	Steps: 451800
Train Epoch: 4 [12800/32259 (40%)]	Loss: 0.222480	Steps: 471800
Train Epoch: 4 [19200/32259 (59%)]	Loss: 0.200346	Steps

KeyboardInterrupt: 

In [113]:
from sklearn.metrics import classification_report
def get_metrics(dataLoader):
    total = 0
    correct = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in dataLoader: # just 1 batch
            model.eval()
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            total+=target.size(0)
            correct+=(predicted == target).sum().item()
            report = classification_report(target, predicted, output_dict=True)
            precision += report['macro avg']['precision']
            recall += report['macro avg']['recall']
            f1_score += report['macro avg']['f1-score']
            accuracy += report['accuracy']
            print(report)
    print("Precision: {}, Recall: {}, F1-Score: {}, Accuracy: {}, AccuracyCust: {}".format(precision, recall, f1_score, accuracy, correct/total))
    

In [114]:
fileNameVal = 'Ds.csv'
datasetVal = NSLKDDDataset(fileNameVal)
params = {'batch_size': 22544, 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
#RuntimeError: expected scalar type Int but found Float
#get_metrics(dataGeneratorVal)
def val():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorVal:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorVal.dataset),
            100. * correct / len(dataGeneratorVal.dataset)))
        return test_loss

val()


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:16: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  app.launch_new_instance()


tensor([[0],
        [1],
        [1],
        ...,
        [1],
        [1],
        [0]])

Test set: Average loss: 0.0000, Accuracy: 18147/22544 (80%)



0.0

In [115]:
get_metrics(dataGeneratorVal)

{'0': {'precision': 0.7130713712910987, 'recall': 0.9156626506024096, 'f1-score': 0.8017672783012488, 'support': 9711}, '1': {'precision': 0.9187016081000595, 'recall': 0.7211875633133328, 'f1-score': 0.8080499410660495, 'support': 12833}, 'accuracy': 0.8049591909155429, 'macro avg': {'precision': 0.8158864896955791, 'recall': 0.8184251069578712, 'f1-score': 0.8049086096836492, 'support': 22544}, 'weighted avg': {'precision': 0.8301248147336729, 'recall': 0.8049591909155429, 'f1-score': 0.8053436361463822, 'support': 22544}}
Precision: 0.8158864896955791, Recall: 0.8184251069578712, F1-Score: 0.8049086096836492, Accuracy: 0.8049591909155429, AccuracyCust: 0.8049591909155429


In [116]:
get_metrics(dataGeneratorTrain)

{'0': {'precision': 0.9375, 'recall': 1.0, 'f1-score': 0.967741935483871, 'support': 30}, '1': {'precision': 1.0, 'recall': 0.9411764705882353, 'f1-score': 0.9696969696969697, 'support': 34}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.96875, 'recall': 0.9705882352941176, 'f1-score': 0.9687194525904204, 'support': 64}, 'weighted avg': {'precision': 0.970703125, 'recall': 0.96875, 'f1-score': 0.9687805474095796, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 39}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 25}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 0.9375, 'recall': 0.967741935483871, 'f1-score': 0.9523809523809523, 'support': 31}, '1': {'precision': 0.96875, 'recall': 0.9393939393939394, 'f1-score': 0.9538461538461539, 'support': 33}, 'accuracy': 0.953125, 'macr

{'0': {'precision': 0.9696969696969697, 'recall': 1.0, 'f1-score': 0.9846153846153847, 'support': 32}, '1': {'precision': 1.0, 'recall': 0.96875, 'f1-score': 0.9841269841269841, 'support': 32}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9848484848484849, 'recall': 0.984375, 'f1-score': 0.9843711843711844, 'support': 64}, 'weighted avg': {'precision': 0.9848484848484849, 'recall': 0.984375, 'f1-score': 0.9843711843711844, 'support': 64}}
{'0': {'precision': 0.9411764705882353, 'recall': 0.9696969696969697, 'f1-score': 0.955223880597015, 'support': 33}, '1': {'precision': 0.9666666666666667, 'recall': 0.9354838709677419, 'f1-score': 0.9508196721311476, 'support': 31}, 'accuracy': 0.953125, 'macro avg': {'precision': 0.9539215686274509, 'recall': 0.9525904203323559, 'f1-score': 0.9530217763640814, 'support': 64}, 'weighted avg': {'precision': 0.9535232843137256, 'recall': 0.953125, 'f1-score': 0.9530905921213605, 'support': 64}}
{'0': {'precision': 0.9655172413793104, 'recall': 0.

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 36}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 28}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.9655172413793104, 'f1-score': 0.9824561403508771, 'support': 29}, '1': {'precision': 0.9722222222222222, 'recall': 1.0, 'f1-score': 0.9859154929577464, 'support': 35}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9861111111111112, 'recall': 0.9827586206896552, 'f1-score': 0.9841858166543118, 'support': 64}, 'weighted avg': {'precision': 0.9848090277777778, 'recall': 0.984375, 'f1-score': 0.9843479738077587, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 38}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 26}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1

{'0': {'precision': 0.9736842105263158, 'recall': 1.0, 'f1-score': 0.9866666666666666, 'support': 37}, '1': {'precision': 1.0, 'recall': 0.9629629629629629, 'f1-score': 0.9811320754716981, 'support': 27}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.986842105263158, 'recall': 0.9814814814814814, 'f1-score': 0.9838993710691823, 'support': 64}, 'weighted avg': {'precision': 0.9847861842105263, 'recall': 0.984375, 'f1-score': 0.9843317610062892, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 31}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.96875, 'f1-score': 0.9841269841269841, 'support': 32}, '1': {'precision': 0.9696969696969697, 'recall': 1.0, 'f1-score': 0.9846153846153847, 'support': 32}, 'accuracy': 

{'0': {'precision': 1.0, 'recall': 0.9705882352941176, 'f1-score': 0.9850746268656716, 'support': 34}, '1': {'precision': 0.967741935483871, 'recall': 1.0, 'f1-score': 0.9836065573770492, 'support': 30}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9838709677419355, 'recall': 0.9852941176470589, 'f1-score': 0.9843405921213604, 'support': 64}, 'weighted avg': {'precision': 0.9848790322580645, 'recall': 0.984375, 'f1-score': 0.9843864692928799, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 31}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 0.96875, 'recall': 1.0, 'f1-score': 0.9841269841269841, 'support': 31}, '1': {'precision': 1.0, 'recall': 0.9696969696969697, 'f1-score': 0.9846153846153847, 'support': 33}, 'accuracy': 

...

{'0': {'precision': 1.0, 'recall': 0.9705882352941176, 'f1-score': 0.9850746268656716, 'support': 34}, '1': {'precision': 0.967741935483871, 'recall': 1.0, 'f1-score': 0.9836065573770492, 'support': 30}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9838709677419355, 'recall': 0.9852941176470589, 'f1-score': 0.9843405921213604, 'support': 64}, 'weighted avg': {'precision': 0.9848790322580645, 'recall': 0.984375, 'f1-score': 0.9843864692928799, 'support': 64}}
{'0': {'precision': 0.972972972972973, 'recall': 0.9473684210526315, 'f1-score': 0.9599999999999999, 'support': 38}, '1': {'precision': 0.9259259259259259, 'recall': 0.9615384615384616, 'f1-score': 0.9433962264150944, 'support': 26}, 'accuracy': 0.953125, 'macro avg': {'precision': 0.9494494494494494, 'recall': 0.9544534412955465, 'f1-score': 0.951698113207547, 'support': 64}, 'weighted avg': {'precision': 0.9538601101101101, 'recall': 0.953125, 'f1-score': 0.9532547169811321, 'support': 64}}
{'0': {'precision': 1.0, 'recall'

{'0': {'precision': 0.8947368421052632, 'recall': 1.0, 'f1-score': 0.9444444444444444, 'support': 34}, '1': {'precision': 1.0, 'recall': 0.8666666666666667, 'f1-score': 0.9285714285714286, 'support': 30}, 'accuracy': 0.9375, 'macro avg': {'precision': 0.9473684210526316, 'recall': 0.9333333333333333, 'f1-score': 0.9365079365079365, 'support': 64}, 'weighted avg': {'precision': 0.944078947368421, 'recall': 0.9375, 'f1-score': 0.9370039682539681, 'support': 64}}
{'0': {'precision': 0.9629629629629629, 'recall': 0.9629629629629629, 'f1-score': 0.9629629629629629, 'support': 27}, '1': {'precision': 0.972972972972973, 'recall': 0.972972972972973, 'f1-score': 0.972972972972973, 'support': 37}, 'accuracy': 0.96875, 'macro avg': {'precision': 0.967967967967968, 'recall': 0.967967967967968, 'f1-score': 0.967967967967968, 'support': 64}, 'weighted avg': {'precision': 0.96875, 'recall': 0.96875, 'f1-score': 0.96875, 'support': 64}}
{'0': {'precision': 0.967741935483871, 'recall': 0.9375, 'f1-scor

{'0': {'precision': 0.9642857142857143, 'recall': 1.0, 'f1-score': 0.9818181818181818, 'support': 27}, '1': {'precision': 1.0, 'recall': 0.972972972972973, 'f1-score': 0.9863013698630138, 'support': 37}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9821428571428572, 'recall': 0.9864864864864865, 'f1-score': 0.9840597758405978, 'support': 64}, 'weighted avg': {'precision': 0.9849330357142857, 'recall': 0.984375, 'f1-score': 0.9844100249066002, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 33}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 31}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 0.9142857142857143, 'recall': 0.9411764705882353, 'f1-score': 0.9275362318840579, 'support': 34}, '1': {'precision': 0.9310344827586207, 'recall': 0.9, 'f1-score': 0.9152542372881356, 's

{'0': {'precision': 0.9705882352941176, 'recall': 1.0, 'f1-score': 0.9850746268656716, 'support': 33}, '1': {'precision': 1.0, 'recall': 0.967741935483871, 'f1-score': 0.9836065573770492, 'support': 31}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9852941176470589, 'recall': 0.9838709677419355, 'f1-score': 0.9843405921213604, 'support': 64}, 'weighted avg': {'precision': 0.9848345588235294, 'recall': 0.984375, 'f1-score': 0.9843635307071201, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 34}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 30}, 'accuracy': 1.0, 'macro avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}, 'weighted avg': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.9696969696969697, 'f1-score': 0.9846153846153847, 'support': 33}, '1': {'precision': 0.96875, 'recall': 1.0, 'f1-score': 0.9841269841269841, 'support': 31}, 'accuracy': 

{'0': {'precision': 1.0, 'recall': 0.9583333333333334, 'f1-score': 0.9787234042553191, 'support': 24}, '1': {'precision': 0.975609756097561, 'recall': 1.0, 'f1-score': 0.9876543209876543, 'support': 40}, 'accuracy': 0.984375, 'macro avg': {'precision': 0.9878048780487805, 'recall': 0.9791666666666667, 'f1-score': 0.9831888626214866, 'support': 64}, 'weighted avg': {'precision': 0.9847560975609756, 'recall': 0.984375, 'f1-score': 0.9843052272130286, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 0.9142857142857143, 'f1-score': 0.955223880597015, 'support': 35}, '1': {'precision': 0.90625, 'recall': 1.0, 'f1-score': 0.9508196721311475, 'support': 29}, 'accuracy': 0.953125, 'macro avg': {'precision': 0.953125, 'recall': 0.9571428571428571, 'f1-score': 0.9530217763640813, 'support': 64}, 'weighted avg': {'precision': 0.95751953125, 'recall': 0.953125, 'f1-score': 0.9532282236359189, 'support': 64}}
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 34}, '1': {'precisio